In [ ]:
# default_exp compute_metrics

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from grade_classif.params.parser import hparams
from grade_classif.imports import *
from grade_classif.models.plmodules import Normalizer
from grade_classif.data.read import get_items
from grade_classif.data.color import rgb_to_lab

In [ ]:
hparams.open_mode = 'H'
hparams.transforms = 1
hparams.normalizer = 'cbr_5_32_4'
#hparams.loss = 'mse'
hparams.size = 299
hparams.gpus = [1]
hparams.filt = 'all'
hparams.batch_size = 16
hparams.patch_classes = None
hparams.norm_csv = None
hparams.concept_classes = None
hparams.data = Path('/data/DeepLearning/SCHWOB_Robin/Patches_299/Patches_299_1')
hparams.data_csv = Path('/home/DeepLearning/grade_classif/csvs/scans.csv')

In [ ]:
#df = pd.read_csv(hparams.patch_classes, index_col='patchId')
def get_filt(ds):
    def filt(x):
        return ds in x.stem
    return filt

In [ ]:
# export
norm = Normalizer(hparams)

In [ ]:
# export
all_items = norm.data.train._ds.items
all_labels = norm.data.train._ds.labels

In [ ]:
# export
norm.load('6cacb7558df8464a843775ca6694ed3b')

In [ ]:
# export
norm.freeze()

In [ ]:
# export
ref = Path('/data/DeepLearning/SCHWOB_Robin/Patches_normacolor_1024/Patches_1024_1/CF_Normacolor_0182')

In [ ]:
# export
ref_items, _ = get_items(ref, lambda x: x, extensions=['.png'])

In [ ]:
# export
norm.data.valid._ds.items = ref_items
norm.data.valid._ds.labels = ref_items

In [ ]:
# export
ref_means = torch.zeros(3, device=norm.main_device)
ref_stds = torch.zeros(3, device=norm.main_device)

In [ ]:
# export
i = 0
for _, x in norm.val_dataloader()[0]:
    x = rgb_to_lab(x.to(norm.main_device))
    x[:, 0] = x[:, 0] / 100
    x[:, 1:] = (x[:, 1:] + 127) / 254
    ref_means += x.mean((0, 2, 3))
    ref_stds += x.std((0, 2, 3))
    i += 1
ref_means /= i
ref_stds /= i

In [ ]:
ref_means, ref_stds

(tensor([0.7834, 0.5751, 0.4766], device='cuda:0'),
 tensor([0.1473, 0.0481, 0.0438], device='cuda:0'))

In [ ]:
# export
norm.data.valid._ds.items = all_items
norm.data.valid._ds.labels = all_labels

In [ ]:
# export
scans = np.array(list(map(lambda x: x.parent.name, all_items)))

In [ ]:
# export
scan_names = np.unique(scans)

In [ ]:
# export
scan_indices = np.zeros(len(scans), dtype=np.int32)
counts = np.zeros(len(scan_names), dtype=np.int32)

In [ ]:
# export
for i in range(len(scan_names)):
    t = scans == scan_names[i]
    scan_indices[t] = i
    counts[i] = t.sum()

In [ ]:
# export
alpha_means_base = torch.zeros(len(scan_names), device=norm.main_device)
beta_means_base = torch.zeros(len(scan_names), device=norm.main_device)
alpha_means = torch.zeros(len(scan_names), device=norm.main_device)
beta_means = torch.zeros(len(scan_names), device=norm.main_device)
alpha_stds = torch.zeros(len(scan_names), device=norm.main_device)
beta_stds = torch.zeros(len(scan_names), device=norm.main_device)
ssims = torch.zeros(len(scan_names), device=norm.main_device)
rhos = torch.zeros(len(scan_names), device=norm.main_device)

In [ ]:
# export
bs = norm.hparams.batch_size

In [ ]:
# export
dl = norm.val_dataloader()[0]

In [ ]:
# export
def wrapper(gen):
    while True:
        try:
            yield(next(gen))
        except StopIteration:
            break
        except Exception as e:
            print(e)
            continue

In [ ]:
# export
for k, (y, x) in tqdm(enumerate(wrapper(iter(dl))), total=len(dl)):
    x = rgb_to_lab(x.to(norm.main_device))
    x[:, 0] = x[:, 0] / 100
    x[:, 1:] = (x[:, 1:] + 127) / 254
    y = rgb_to_lab(norm.predict(y.to(norm.main_device)))
    y[:, 0] = y[:, 0] / 100
    y[:, 1:] = (y[:, 1:] + 127) / 254
    idxs = scan_indices[bs*k: bs*(k+1)]
    mu_x = x.mean((2, 3))
    std_x = x.std((2, 3))
    mu_y = y.mean((2, 3))
    std_y = y.std((2, 3))
    mu_xy = (x*y).mean((2, 3))
    std_xy = mu_xy - mu_x*mu_y
    for idx in np.unique(idxs):
        t = idxs == idx
        alpha_means_base[idx] += mu_x[t, 1].sum(0)
        beta_means_base[idx] += mu_x[t, 2].sum(0)
        alpha_means[idx] += mu_y[t, 1].sum(0)
        beta_means[idx] += mu_y[t, 2].sum(0)
        alpha_stds[idx] += std_y[t, 1].sum(0)
        beta_stds[idx] += std_y[t, 2].sum(0)
        ssims[idx] += ((2*mu_x[t, 0]*mu_y[t, 0]+1e-2)*(2*std_xy[t, 0]+1e-2)/((mu_x[t, 0]**2+mu_y[t, 0]**2+1e-2)*(std_x[t, 0]**2+std_y[t, 0]**2+1e-2))).sum(0)
        rhos[idx] += (std_xy[t, 0]/(std_x[t, 0]*std_y[t, 0])).sum(0)

In [ ]:
# export
alpha_means_base = alpha_means_base.cpu()
alpha_means = alpha_means.cpu()
beta_means = beta_means.cpu()
beta_means_base = beta_means_base.cpu()
alpha_stds = alpha_stds.cpu()
beta_stds = beta_stds.cpu()
ssims = ssims.cpu()
rhos = rhos.cpu()

In [ ]:
# export
alpha_means_base /= counts
beta_means_base /= counts
alpha_means /= counts
beta_means /= counts
alpha_stds /= counts
beta_stds /= counts
ssims /= counts
rhos /= counts

In [ ]:
df = pd.DataFrame({'Scan_Id': scan_names, 'alpha_means_base': alpha_means_base,
                   'alpha_means': alpha_means, 'beta_means_base': beta_means_base,
                   'beta_means': beta_means, 'alpha_stds': alpha_stds,
                   'beta_stds': beta_stds, 'ssims': ssims,
                   'pccs': rhos})

In [ ]:
df.head()

,Scan_Id,alpha_means_base,alpha_means,beta_means_base,beta_means,alpha_stds,beta_stds,ssims,pccs
0,CF_PACS04rescan_06P008,0.544102,0.569136,0.469929,0.483457,0.028289,0.023617,0.915664,0.884638
1,CF_PACS04rescan_06P017,0.562141,0.569465,0.460488,0.478691,0.032650,0.024084,0.897253,0.858841
2,CF_PACS04rescan_06P024,0.578713,0.571932,0.449322,0.476591,0.024573,0.025572,0.903867,0.870045
3,CF_PACS04rescan_06P027,0.546032,0.552719,0.462103,0.472768,0.027289,0.029663,0.947586,0.930893
4,CF_PACS04rescan_06P028,0.558598,0.558709,0.458466,0.477708,0.026208,0.024257,0.910813,0.868972


In [ ]:
df.to_csv(hparams.data_csv.parent/'norm_metrics_train.csv', index=False, sep=';')

In [ ]:
1

1

In [ ]:
# export
print(f'AMCE_a base : {torch.abs(alpha_means_base-ref_means[1]).mean()}')
print(f'AMCE_b base: {torch.abs(beta_means_base-ref_means[2]).mean()}')
print(f'AMCE_a : {torch.abs(alpha_means-ref_means[1]).mean()}')
print(f'AMCE_b : {torch.abs(beta_means-ref_means[2]).mean()}')
print(f'ADCE_a : {torch.abs(alpha_stds-ref_stds[1]).mean()}')
print(f'ADCE_b : {torch.abs(beta_stds-ref_stds[2]).mean()}')
print(f'MSSIM : {ssims.mean()}')
print(f'PCC : {rhos.mean()}')

AMCE_a base : 0.03058633086833897
AMCE_b base: 0.02770463004652188
AMCE_a : 0.0107072311397508
AMCE_b : 0.008540492600498602
ADCE_a : 0.019733678567921176
ADCE_b : 0.02000269205375632
MSSIM : 0.8964512756231039
PCC : 0.855185212461656


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script(fname='_compute_metrics.ipynb')